In [4]:
import mido
from mido import MidiFile, MidiTrack, Message, MetaMessage
from os import listdir
from os.path import isfile, split, join
import argparse

In [5]:
def split_midi_into_15_second_parts(src_midi_path='data/midis/enchanted.mid', dest_midi_base_path='data/short_midis/short_midis'):
    src_midi = MidiFile(src_midi_path)

    # Assuming a default tempo of 500,000 microseconds per beat
    # Adjust this if your MIDI file specifies a different initial tempo
    default_tempo = 500000
    ticks_per_second = src_midi.ticks_per_beat * (default_tempo / 1000000)
    ticks_for_2_seconds = ticks_per_second * 15

    current_segment = 0
    current_ticks_in_segment = 0

    # Initialize the first segment MIDI file
    segment_midi = MidiFile()
    segment_midi.ticks_per_beat = src_midi.ticks_per_beat

    for track in src_midi.tracks:
        new_track = MidiTrack()
        segment_midi.tracks.append(new_track)
        for msg in track:
            # Adjust for tempo changes if the message is a tempo change
            if msg.type == 'set_tempo':
                ticks_per_second = src_midi.ticks_per_beat * (msg.tempo / 1000000)
                ticks_for_2_seconds = ticks_per_second * 15

            # Check if adding the current message would exceed the 2-second limit for this segment
            if current_ticks_in_segment + msg.time > ticks_for_2_seconds:
                # Save the current segment MIDI file
                segment_midi.save(f'{dest_midi_base_path}_part_{current_segment}.mid')

                # Start a new segment MIDI file
                segment_midi = MidiFile()
                segment_midi.ticks_per_beat = src_midi.ticks_per_beat
                new_track = MidiTrack()
                segment_midi.tracks.append(new_track)

                # Reset the tick count for the new segment and increment the segment counter
                current_ticks_in_segment = 0
                current_segment += 1

            # Add the current message to the track and update the tick count
            new_track.append(msg)
            current_ticks_in_segment += msg.time

    # Save the last segment if it has any content
    if current_ticks_in_segment > 0:
        segment_midi.save(f'{dest_midi_base_path}_part_{current_segment}.mid')

# Example usage


In [6]:
split_midi_into_15_second_parts()